<a href="https://colab.research.google.com/github/Jakki-Metabolism/Milestone01-0531/blob/main/Food_Recognition_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

In [ ]:
# Let's make sure the kaggle.json file is present.
!ls -lha kaggle.json

-rw-r--r-- 1 root root 62 May 29 06:15 kaggle.json


In [ ]:
!pip install -q kaggle

In [ ]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Copy the stackoverflow data set locally.
!kaggle datasets download -d kmader/food41

food41.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
#unzip the dataset
!unzip \*.zip  && rm *.zip

Archive:  food41.zip
replace food_c101_n1000_r384x384x3.h5? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Activation
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import scipy
import cv2

import os
from os import listdir
from os.path import isfile, join
import shutil
import stat
import collections
from collections import defaultdict

In [ ]:
!ls

 food41.zip			      food_test_c101_n1000_r64x64x1.h5
 food_c101_n1000_r384x384x3.h5	      food_test_c101_n1000_r64x64x3.h5
 food_c101_n10099_r32x32x1.h5	      images
 food_c101_n10099_r32x32x3.h5	     'kaggle (1).json'
 food_c101_n10099_r64x64x1.h5	     'kaggle (2).json'
 food_c101_n10099_r64x64x3.h5	      kaggle.json
 food_test_c101_n1000_r128x128x1.h5   meta
 food_test_c101_n1000_r128x128x3.h5   sample_data
 food_test_c101_n1000_r32x32x1.h5     test
 food_test_c101_n1000_r32x32x3.h5     train


In [ ]:
#divide fold_images into train and test - two folds
if not os.path.isdir('./test') and not os.path.isdir('./train'):

    def copytree(src, dst, symlinks = False, ignore = None):
        if not os.path.exists(dst):
            os.makedirs(dst)
            shutil.copystat(src, dst)
        lst = os.listdir(src)
        if ignore:
            excl = ignore(src, lst)
            lst = [x for x in lst if x not in excl]
        for item in lst:
            s = os.path.join(src, item)
            d = os.path.join(dst, item)
            if symlinks and os.path.islink(s):
                if os.path.lexists(d):
                    os.remove(d)
                os.symlink(os.readlink(s), d)
                try:
                    st = os.lstat(s)
                    mode = stat.S_IMODE(st.st_mode)
                    os.lchmod(d, mode)
                except:
                    pass # lchmod not available
            elif os.path.isdir(s):
                copytree(s, d, symlinks, ignore)
            else:
                shutil.copy2(s, d)

    def generate_dir_file_map(path):
        dir_files = defaultdict(list)
        with open(path, 'r') as txt:
            files = [l.strip() for l in txt.readlines()]
            for f in files:
                dir_name, id = f.split('/')
                dir_files[dir_name].append(id + '.jpg')
        return dir_files

    train_dir_files = generate_dir_file_map('/content/test')
    test_dir_files = generate_dir_file_map('/content/train')


    def ignore_train(d, filenames):
        print(d)
        subdir = d.split('/')[-1]
        to_ignore = train_dir_files[subdir]
        return to_ignore

    def ignore_test(d, filenames):
        print(d)
        subdir = d.split('/')[-1]
        to_ignore = test_dir_files[subdir]
        return to_ignore

    copytree('./images', './train', ignore=ignore_test)
    copytree('./images', './test', ignore=ignore_train)
    
    
else:
    print('Train/Test files already copied into separate folders.')

Train/Test files already copied into separate folders.


In [ ]:
#config
img_width, img_height = 256, 256
input_depth = 3
train_data_dir = './train'
testing_data_dir = './test'
epochs = 5
batch_size = 5

train_datagen = ImageDataGenerator(rescale=1/255)
test_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    color_mode="rgb",
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    seed=42)

testing_generator = test_datagen.flow_from_directory(
    testing_data_dir,
    color_mode="rgb",
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    seed=42)

NUMB_FILTER_L1 = 1000
NUMB_FILTER_L2 = 1000
NUMB_FILTER_L3 = 1000
NUMB_FILTER_L4 = 500
NUMB_NODE_FC_LAYER = 1000

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


In [ ]:
if K.image_data_format() == 'channels_first':
    input_shape_val = (input_depth, img_width, img_height)
else:
    input_shape_val = (img_width, img_height, input_depth)

model = Sequential()

#layer 1
model.add(Conv2D(NUMB_FILTER_L1, (3,3),
                 input_shape = input_shape_val,
                 padding='same', name='input_sensor'))
model.add(Activation('relu'))
model.add(MaxPooling2D((2,2)))

#layer2
model.add(Conv2D(NUMB_FILTER_L3, (3,3),
                 padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D((2,2)))

#layer 3
model.add(Conv2D(NUMB_FILTER_L4, (3,3),
                 padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D((2,2)))

#flatten the model for fully connected layer
model.add(Flatten())

#fully connected layer
model.add(Dense(NUMB_NODE_FC_LAYER,activation='relu'))

#output layer
model.add(Dense(train_generator.num_classes,
                activation='softmax', name='output_generator'))

#compile the network
model.compile(loss='categorical_crossentropy',
              optimizer='sgd', metrics=['accuracy'])

In [ ]:
#show the model summary
model.summary()

#train the network
model.fit(
    train_generator,
    steps_per_epoch=np.floor(train_generator.n/batch_size),
    epochs=epochs,
    validation_data=testing_generator,
    validation_steps=np.floor(testing_generator.n/batch_size)
)

print("training is done!")
model.save('/content/model')
print('model is successfully saved!')

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_sensor (Conv2D)        (None, 256, 256, 1000)    28000     
_________________________________________________________________
activation_3 (Activation)    (None, 256, 256, 1000)    0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 128, 128, 1000)    0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 128, 1000)    9001000   
_________________________________________________________________
activation_4 (Activation)    (None, 128, 128, 1000)    0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 64, 64, 1000)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 500)      

ValueError: ignored

In [ ]:
model.evaluate(
    testing_generator,
    np.floor(testing_generator.n/batch_size)
)

NameError: ignored

In [ ]:
print("training is done!")
model.save('/content/model')
print('model is successfully saved!')